<a href="https://colab.research.google.com/github/aeapolimi/duckietown/blob/master/duckietown_stablebaseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Train duckietown with stable baseline

Environment setup 

In [ ]:
import os 
if not os.path.isdir('gym-duckietown') and not os.path.isdir('../gym-duckietown'):
  branch = "master" #@param ['master', 'daffy']
  !git clone --branch {branch} https://github.com/duckietown/gym-duckietown.git
  !pip3 install -e gym-duckietown
if "/gym-duckietown" not in os.getcwd():
  os.chdir('gym-duckietown')
!apt install xvfb -y
!pip3 install pyvirtualdisplay
from pyvirtualdisplay import Display
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
%matplotlib inline

Install stable baseline

In [ ]:
# Stable baseline requires tensorflow 1.x
%tensorflow_version 1.x

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
!apt update && apt install cmake libopenmpi-dev python3-dev zlib1g-dev
!pip3 install stable-baselines[mpi]
#This version with mpi is apparently broken, see:
#https://github.com/hill-a/stable-baselines/issues/464
!pip3 install stable-baselines --upgrade

In [ ]:
map_name = "Duckietown-small_loop-v0" #@param ['Duckietown-straight_road-v0','Duckietown-4way-v0','Duckietown-udem1-v0','Duckietown-small_loop-v0','Duckietown-small_loop_cw-v0','Duckietown-zigzag_dists-v0','Duckietown-loop_obstacles-v0','Duckietown-loop_pedestrians-v0']

Wrapper to highlight lines

In [ ]:
import cv2
import gym
from gym.spaces import Box

cutted_img_height = 350 #@param {type: "slider", min: 0, max: 480, step:1}
resize_ratio = 0.35 #@param {type: "slider", min: 0.0, max: 1.0, step:0.01}

img_height = 480
top_crop = img_height - cutted_img_height

img_final_height = int(cutted_img_height * resize_ratio)
img_final_width = int(640 * resize_ratio)

def cropimg(img):
    """
    Crop top of image top_crop px, they are noise most of the time

    :param img: (RGB image as np array) Image to be cropped
    """
    return img[top_crop:,:]

def houghtransform(img):
    """
    Apply Hough Line transform, for theory see:
    https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_houghlines/py_houghlines.html

    :param img: (RGB image as np array)
    """
    frame_BGR = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    #gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY, 3)
    edges = cv2.Canny(frame_BGR,50,150,apertureSize = 3)
    #minLineLength = 100
    #maxLineGap = 10
    #lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength,maxLineGap)
    #for x1,y1,x2,y2 in lines[0]:
    #    cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
    imgRGB = cv2.cvtColor(edges, cv2.COLOR_BGR2RGB)
    return imgRGB

def resizeimg(img, ratio):
    """
    Resize image
    :param img: (np array)
    :param ratio: (float) 0<ratio<1
    """
    return cv2.resize(img, (0,0), fx=ratio, fy=ratio) 
  
def takeyellow(img):
    """
    Extract yellow lines, for color ranges see:
    https://stackoverflow.com/questions/48109650/how-to-detect-two-different-colors-using-cv2-inrange-in-python-opencv

    :param img: (RGB image as np array)
    """
    frame_HSV = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    frame_threshold = cv2.inRange(frame_HSV, (20,100,100), (50, 255, 255))
    imgRGB = cv2.cvtColor(frame_threshold, cv2.COLOR_GRAY2RGB)
    return imgRGB

def takewhiteyellow(img):
    """
    Extract white and yellow lines

    :param img: (RGB image as np array)
    """
    #white
    sensitivity = 100
    lower_white = np.array([0,0,255-sensitivity])
    upper_white = np.array([255,sensitivity,255])
    frame_HSV = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    maskwhite = cv2.inRange(frame_HSV, lower_white, upper_white)
    img[maskwhite > 0] = (255, 0, 0)
    img[maskwhite == 0] = (0,0,0)
    #yellow
    maskyellow = cv2.inRange(frame_HSV, (15,70,70), (50, 255, 255))
    img[maskyellow > 0] = (0, 255, 0)
    return img

def white_balance(img):
    """
    Grayworld assumption:
    https://stackoverflow.com/questions/46390779/automatic-white-balancing-with-grayworld-assumption/46391574

    :param img: (RGB image as np array)
    """
    result = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result = cv2.cvtColor(result, cv2.COLOR_LAB2RGB)
    return result

class ObsWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(ObsWrapper, self).__init__(env)
        self.observation_space = Box(0, 255, (img_final_height, img_final_width, 3), dtype=self.observation_space.dtype)
        self.accept_start_angle_deg = 4
        self.env = env

    def observation(self, obs):
        cropped = cropimg(obs)
        resized = resizeimg(cropped, resize_ratio)
        balanced = white_balance(resized)
        img = takewhiteyellow(balanced)
        return img

In [ ]:
class NormalizeWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(NormalizeWrapper, self).__init__(env)
        self.observation_space = Box(0, 1, self.observation_space.shape, dtype=self.observation_space.dtype)
        self.env = env

    def observation(self, obs):
        obs = obs/255
        return obs

Wrapper result example:

In [ ]:
import gym_duckietown

display = Display(visible=0, size=(1400, 900))
display.start()
env = gym.make(map_name, accept_start_angle_deg=4)
env = ObsWrapper(env)
plt.imshow(env.reset())

In [ ]:
class DtRewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super(DtRewardWrapper, self).__init__(env)

    def reward(self, reward):
        if reward == -1000:
            reward = -10
        elif reward > 0:
            reward += 10
        else:
            reward += 4

        return reward

In [ ]:
env = DtRewardWrapper(env)

In [ ]:
env.observation_space

In [ ]:
env.action_space

In [ ]:
%tensorboard --logdir ./a2c_duckieloop/

NB Con lr = 0.0007 e gamma >= 0.6 reward converge a -1000, con 0.5 esce la situazione reward = 0 e la macchina fa retromarcia invece di curvare

Con lr = 0.007 e gamma = 0.5 converge a -1000 nei primi 30k (CnnLstm)

Con lr = 0.00007 e gamma 0.5 fa ancora retromarcia invece di curvare con 200k, con gamma 0.6 e 0.05 uguale. (CnnLstm)

Con lr = 0.00007 e gamma 0.4 fa ancora retromarcia ma curva un po' curvare con 200k, ma i mean reward nel train sono piuttosto alti (>-600). (CnnLstm)

0.3 come 0.4 ma reward non buoni

0.1 schifo

0.2 non fa neanche i rettilinei

In [ ]:
from stable_baselines.common.vec_env import VecFrameStack
from stable_baselines import A2C
from stable_baselines.common.policies import CnnLstmPolicy
from stable_baselines.common.evaluation import evaluate_policy
 
model = A2C(
    CnnLstmPolicy,
    env,
    gamma=0.337338101, #Discount reward def=0.99
    n_steps=5,
    learning_rate=0.0001435, #def=0.0007
    lr_schedule='constant',
    verbose=0,
    tensorboard_log="./a2c_duckieloop/"
    )
 
# Change range to train more
if True:
  print("Starting...")
  for time in range(10):
    model.learn(total_timesteps=int(2e4))
    model.save("a2c"+map_name+str(1e4*time))
    mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
    print(f"#{time} Trained 10000 timesteps, mean_reward: {mean_reward}, std_reward: {std_reward}")
  
  ipythondisplay.clear_output(wait=True)

In [ ]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [ ]:
from stable_baselines.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: DtRewardWrapper(ObsWrapper(gym.make(map_name, accept_start_angle_deg=4)))])
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, reward, done, _ = eval_env.step(action)
    if done and reward < 0:
      print("*** CRASHED ***")
      print(reward)
    elif done:
      print("SAFE")
      print(reward)
    

  # Close the video recorder
  eval_env.close()

In [ ]:
if False:
    time = 4
    model_to_be_loaded = "a2c"+map_name+str(1e4*time)
    model = A2C.load(model_to_be_loaded)
if True:
  record_video(model, video_length=500, prefix=map_name)

In [ ]:
if True:
  show_videos('videos', prefix=map_name)

In [ ]:
# To continue the training set to True
if False:
  # To load a model set to True
  if False:
    from stable_baselines import A2C
    model_to_be_loaded = "a2c"+map_name+""
    model = A2C.load(model_to_be_loaded, tensorboard_log="./a2c_duckieloop/")
    model.set_env(env)
  for time in range(5):
    model.learn(total_timesteps=int(1e5))
    model.save("a2c"+map_name+str(1e5*time))
    mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
    print(f"Trained 100000 timesteps, mean_reward: {mean_reward}, std_reward: {std_reward}")

Test parameter search using optuna:

Risultati test gamma:

Promettente intorno a 0.565 - 0.567

0.567113528 -> -149

0.418975447 -> -202

Buono intorno a 0.33 - 0.337 ma con uno strano picco negativo a 0.336

0.337338101 -> -28

In [ ]:
def objective(trial):

    gamma = 0.567113528
    lr = trial.suggest_float("learning_rate", 0.00005, 0.005, log=True)

    model = A2C(
      CnnLstmPolicy,
      env,
      gamma=gamma,
      n_steps=5,
      learning_rate=lr, #def=0.0007
      lr_schedule='constant',
      verbose=0,
      tensorboard_log="./a2c_duckieloop/"
    )

    model.learn(total_timesteps=int(5e4))
    mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=100)
    print(gamma)
    print(mean_reward)

    return mean_reward

if False:
  !pip install optuna
  import optuna
  study = optuna.create_study(direction='maximize')
  study.optimize(objective, n_trials=20)
  print(study.best_params)
  print(study.best_value)